In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import warnings
warnings.filterwarnings("ignore")



In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
train.head()


In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train_cat = train.select_dtypes(include = ['object'])
test_cat = test.select_dtypes(include = ['object'])
train_cat.info()

In [ ]:
train_numeric = train.select_dtypes(include = ['float64', 'int64'])
test_numeric = test.select_dtypes(include = ['float64', 'int64'])
train_numeric.info()

In [ ]:
missing_cat_train = pd.DataFrame(train_cat.isna().sum().sort_values(ascending = False), columns=['missing_cat_total'])
missing_cat_test = pd.DataFrame(test_cat.isna().sum().sort_values(ascending = False), columns = ['missing_cat_total'])
print(missing_cat_train)
print( '-' * 40)
print(missing_cat_test)

In [ ]:
missing_numeric_train = pd.DataFrame(train_numeric.isna().sum().sort_values(ascending = False), columns=['missing_num_total'])
missing_numeric_test = pd.DataFrame(test_numeric.isna().sum().sort_values(ascending = False), columns = ['missing_num_total'])
print(missing_numeric_train)
print('-' * 40)
print(missing_numeric_test)

#### Let's check and try to fill the missing values in categorical data columns. If the percentage of missing values is very large, we can drop the corresponding column.


In [ ]:
missing_cat_percent = (missing_cat_train['missing_cat_total']/ 1460)*100
print(missing_cat_percent)

In [ ]:
missing_num_percent = (missing_numeric_train['missing_num_total']/1460)*100
print(missing_num_percent)

#### So let's drop the first four which are greater than 50 percent missing

In [ ]:
train = train.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'], axis = 1)
train.head()
# do the same for test data
test = test.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'], axis = 1)

In [ ]:
print(train.shape)
print(test.shape)

#### so now 4 columns are dropped. Let's fill the missing values now

In [ ]:
missing_data = train[['GarageType', 'GarageCond', 'GarageQual', 'GarageFinish', 'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond', 'MasVnrType', 'Electrical', 'LotFrontage', 'GarageYrBlt', 'MasVnrArea']]
missing_data.head()

#### If GarageType is missing that could mean there is no Garage, so let's fill those values and the missing values of all the other realetd columns eaqual to 'none'.
#### let's fill basementfintype2, basementexposure, bsmntfintype1, bsmtqual, bsmtcond, masvnrtype, electrical with none.
#### lets fill lotfrontage with mean value
#### lets fill GarageYrBlt with mode
#### lets fill MasVnrArea with mean

In [ ]:
for col in ('GarageType', 'GarageCond', 'GarageQual', 'GarageFinish', 'BsmtFinType2', 'BsmtFinType1', 'BsmtExposure', 'BsmtQual', 'BsmtCond', 'MasVnrType', 'Electrical', 'MasVnrArea'):
    train[col] = train[col].fillna('None')
train['LotFrontage'] = train['LotFrontage'].fillna(train['LotFrontage'].mean())
train['GarageYrBlt'] = train['GarageYrBlt'].fillna(train['GarageYrBlt'].mean())
# do the same for test data
for col in ('GarageType', 'GarageCond', 'GarageQual', 'GarageFinish', 'BsmtFinType2', 'BsmtFinType1', 'BsmtExposure', 'BsmtQual', 'BsmtCond', 'MasVnrType', 'Electrical', 'MasVnrArea'):
    test[col] = test[col].fillna('None')
test['LotFrontage'] = test['LotFrontage'].fillna(test['LotFrontage'].mean())
test['GarageYrBlt'] = test['GarageYrBlt'].fillna(test['GarageYrBlt'].mean())

In [ ]:
print(train.isna().sum().sort_values(ascending = False))
print('*' * 40)
print(test.isna().sum().sort_values(ascending = False))

#### we ahve dealt with all the missing values in the train data. Now lets check for the ones left in test data and fill them

In [ ]:
test_missing_left = pd.DataFrame(test.isna().sum().sort_values(ascending = False), columns = ['test_missing'])
test_missing_left = test_missing_left[test_missing_left['test_missing']>0]
test_missing_left

#### lets fill all these values with 'None', since missing values are just one or two in each.

In [ ]:
for col in ['MSZoning', 'BsmtHalfBath', 'BsmtFullBath', 'Functional', 'Utilities', 'Exterior1st', 'KitchenQual', 'GarageCars','GarageArea', 'BsmtFinSF1','TotalBsmtSF', 'BsmtFinSF2', 'BsmtUnfSF', 'SaleType', 'Exterior2nd']:
    test[col] = test[col].fillna('None')

In [ ]:
print(test.isna().sum().sort_values(ascending=False))

#### NOw lets check the correlation of all the numerical variables with SalePrice

In [ ]:
plt.figure(figsize=(20,1))
sns.heatmap(train.drop('Id', axis=1).corr().sort_values(by = ['SalePrice'], ascending = False).head(1), cmap='coolwarm', annot=True, annot_kws={'size': 8}, fmt = '.2f')

plt.title('Correlation of Numerical Features with the Target', weight = 'bold', fontsize = 18)
plt.xticks(weight='bold')
plt.yticks(weight='bold', color='dodgerblue', rotation=0)

plt.show()

In [ ]:
plt.figure(figsize=(14,12))
sns.set(font_scale=1.5)
sns.heatmap(train.drop('Id', axis=1).corr(), mask=np.triu(np.ones_like(train.drop('Id', axis=1).corr(), dtype=np.bool)), fmt='.2f', cmap='coolwarm', annot=True, cbar_kws={"shrink": .8}, annot_kws={'size':8})
plt.show()

#### So the highly correlated ones are- 'OverallQual', 'GrlivArea', 'TotalBsmtSF','1st floorSF', 'GarageCars' etc.
#### Let's plot some of these with the target variable

In [ ]:
plt.figure(figsize=(16,10))
sns.set_palette('pastel')
plt.subplot(2,3,1)
sns.regplot(x='GrLivArea', y='SalePrice', data=train)
plt.subplot(2,3,2)
sns.boxplot(x='OverallQual', y='SalePrice', data=train)
plt.subplot(2,3,3)
sns.regplot(x='TotalBsmtSF', y='SalePrice', data=train)
plt.subplot(2,3,4)
sns.regplot(x='1stFlrSF', y='SalePrice', data=train)
plt.subplot(2,3,5)
sns.boxplot(x='GarageCars', y='SalePrice', data=train)
plt.subplot(2,3,6)
sns.regplot(x='GarageArea', y='SalePrice', data=train)
plt.subplots_adjust(wspace=0.5)

plt.show()

#### Now lets check the relation of traget variable with categorical variables

In [ ]:
traincat = train.select_dtypes(include = ['object'])
traincat.columns

In [ ]:
plt.figure(figsize=(20,35))
sns.set_palette('pastel')
sns.set_style('darkgrid')
plt.subplot(8,5,1)
sns.violinplot(x='MSZoning', y='SalePrice', data=train)
plt.subplot(8,5,2)
sns.violinplot(x='Street', y='SalePrice', data=train)
plt.subplot(8,5,3)
sns.violinplot(x='LotShape', y='SalePrice', data=train)
plt.subplot(8,5,4)
sns.violinplot(x='LandContour', y='SalePrice', data=train)
plt.subplot(8,5,5)
sns.violinplot(x='Utilities', y='SalePrice', data=train)
plt.subplot(8,5,6)
sns.violinplot(x='LotConfig', y='SalePrice', data=train)
plt.subplot(8,5,7)
sns.violinplot(x='LandSlope', y='SalePrice', data=train)
plt.subplot(8,5,8)
sns.violinplot(x='Neighborhood', y='SalePrice', data=train)
plt.subplot(8,5,9)
sns.violinplot(x='Condition1', y='SalePrice', data=train)
plt.subplot(8,5,10)
sns.violinplot(x='Condition2', y='SalePrice', data=train)
plt.subplot(8,5,11)
sns.violinplot(x='BldgType', y='SalePrice', data=train)
plt.subplot(8,5,12)
sns.violinplot(x='HouseStyle', y='SalePrice', data=train)
plt.subplot(8,5,13)
sns.violinplot(x='RoofStyle', y='SalePrice', data=train)
plt.subplot(8,5,14)
sns.violinplot(x='RoofMatl', y='SalePrice', data=train)
plt.subplot(8,5,15)
sns.violinplot(x='Exterior1st', y='SalePrice', data=train)
plt.subplot(8,5,16)
sns.violinplot(x='Exterior2nd', y='SalePrice', data=train)
plt.subplot(8,5,17)
sns.violinplot(x='MasVnrType', y='SalePrice', data=train)
plt.subplot(8,5,18)
sns.violinplot(x='MasVnrArea', y='SalePrice', data=train)
plt.subplot(8,5,19)
sns.violinplot(x='ExterQual', y='SalePrice', data=train)
plt.subplot(8,5,20)
sns.violinplot(x='ExterCond', y='SalePrice', data=train)
plt.subplot(8,5,21)
sns.violinplot(x='Foundation', y='SalePrice', data=train)
plt.subplot(8,5,22)
sns.violinplot(x='BsmtQual', y='SalePrice', data=train)
plt.subplot(8,5,23)
sns.violinplot(x='BsmtCond', y='SalePrice', data=train)
plt.subplot(8,5,24)
sns.violinplot(x='BsmtExposure', y='SalePrice', data=train)
plt.subplot(8,5,25)
sns.violinplot(x='BsmtFinType1', y='SalePrice', data=train)
plt.subplot(8,5,26)
sns.violinplot(x='BsmtFinType2', y='SalePrice', data=train)
plt.subplot(8,5,27)
sns.violinplot(x='Heating', y='SalePrice', data=train)
plt.subplot(8,5,28)
sns.violinplot(x='HeatingQC', y='SalePrice', data=train)
plt.subplot(8,5,29)
sns.violinplot(x='CentralAir', y='SalePrice', data=train)
plt.subplot(8,5,30)
sns.violinplot(x='Electrical', y='SalePrice', data=train)
plt.subplot(8,5,31)
sns.violinplot(x='KitchenQual', y='SalePrice', data=train)
plt.subplot(8,5,32)
sns.violinplot(x='Functional', y='SalePrice', data=train)
plt.subplot(8,5,33)
sns.violinplot(x='GarageType', y='SalePrice', data=train)
plt.subplot(8,5,34)
sns.violinplot(x='GarageFinish', y='SalePrice', data=train)
plt.subplot(8,5,35)
sns.violinplot(x='GarageQual', y='SalePrice', data=train)
plt.subplot(8,5,35)
sns.violinplot(x='GarageCond', y='SalePrice', data=train)
plt.subplot(8,5,36)
sns.violinplot(x='PavedDrive', y='SalePrice', data=train)
plt.subplot(8,5,37)
sns.violinplot(x='SaleType', y='SalePrice', data=train)
plt.subplot(8,5,38)
sns.violinplot(x='SaleCondition', y='SalePrice', data=train)
plt.subplots_adjust(wspace=0.5, hspace=0.8)

plt.show()

#### More the variation in mean, stronger the corelation with the target variable.
#### So, Mszoning, neighborhood, condition1, condition2, roofmatl, exterquality, basementqual, centralair, kitchenqual, saletype, MasVnrType are strongly corelated with SalePrice. 


  

#### So let's keep the following features for modelling

In [ ]:
features=['OverallQual', 'YearBuilt','YearRemodAdd','TotalBsmtSF','1stFlrSF','GrLivArea','FullBath','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','MSZoning', 'Neighborhood','Condition1', 'Condition2', 'RoofMatl', 'ExterQual','BsmtQual', 'CentralAir', 'KitchenQual','SaleType','MasVnrType', 'SaleCondition','HouseStyle']

In [ ]:
traintest=pd.concat([train, test], axis=0, sort=False)
traintest.head()

In [ ]:
traintest.tail()

In [ ]:
traintest.shape

In [ ]:
train_test = pd.get_dummies(traintest[features])
train_test.head()

In [ ]:
X_train=train_test[0:1460]
X_test=train_test[1460:2919]
y_train= train[['SalePrice']]

In [ ]:
print('Train', X_train.shape)
print('Test', X_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
X_train = my_imputer.fit_transform(X_train)
X_test = my_imputer.fit_transform(X_test)
lr.fit(X_train, y_train)
prediction = lr.predict(X_test)

In [ ]:

print(round(lr.score(X_train, y_train)*100,2))

#mean_squared_error(y_test, prediction, squared=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(X_train, y_train)
y_pred = forest_model.predict(X_test)
print(round(forest_model.score(X_train, y_train)*100,2))

In [ ]:
print(prediction)

In [ ]:
pred=[]
for item in prediction:
    for i in item:
        pred.append(i)
    
print(pred)

In [ ]:
#Code required to submit for competition
output = pd.DataFrame({'Id':pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')['Id'], 'SalePrice': pred})


In [ ]:
output.head()

In [ ]:
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.gridspec as gridspec
import missingno as msno
import scipy.stats as stats 
from scipy.special import boxcox1p
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Create an empty list
pipeline_models = []

# Assign all models into the list
seed = 42
models = [Ridge(tol=10,random_state=seed),
          Lasso(tol=1,random_state=seed),
          RandomForestRegressor(random_state=seed),
          ExtraTreesRegressor(random_state=seed),
          GradientBoostingRegressor(),
          DecisionTreeRegressor(),
          KNeighborsRegressor()]

model_names = ["Ridge","Lasso","RFR","ETR","GBoost_Reg","DT_Reg","KNN_Reg"]

## Assign each model to a pipeline
for name, model in zip(model_names,models):
    pipeline = ("Scaled_"+ name,
                Pipeline([("Scaler",StandardScaler()),
                          (name,model)
                         ]))
    pipeline_models.append(pipeline)

In [ ]:
## Create a dataframe to store all the models' cross validation score
evaluate = pd.DataFrame(columns=["model","cv","std"])


## Encoded dataset
for name,model in pipeline_models:
    kfold = KFold(n_splits=7,shuffle=True,random_state=42)
    cv = cross_val_score(model, X_train, y_train, cv=kfold, n_jobs=-1, scoring="r2")
    
    row = evaluate.shape[0]
    evaluate.loc[row,"model"] = name
    evaluate.loc[row,"cv"] = round(cv.mean(),3)
    evaluate.loc[row,"std"] = "+/- {}".format(round(cv.std(),4))
    
    evaluate = evaluate.sort_values("cv",ascending=False)

In [ ]:
## Visualization
fig, ax = plt.subplots(1,1,sharey=False,figsize=(16,9))

## Encoded dataset
bar = sns.barplot(evaluate["model"], evaluate["cv"],ax=ax,palette = sns.cubehelix_palette(evaluate.shape[0]))
for rec in bar.patches:
    height = rec.get_height()
    ax.text(rec.get_x() + rec.get_width()/2, height*1.02,height,ha="center")
ax.set_title("Cross Validate Score")
ax.set_xticklabels(evaluate["model"].to_list(),rotation =50)